# Data Collection Phase 1: Target Repository Identification

**Objective:** To construct a dataset of active, professionally developed Virtual Reality (VR) projects from the Unity ecosystem for the purpose of analyzing refactoring patterns.

**Methodology:**
This notebook implements the data collection strategy inspired by Nusrat et al. (2021), utilizing the GitHub API to identify repositories that meet strict criteria for industrial relevance and active maintenance.

**Criteria for Inclusion:**
We filter repositories based on three primary academic metrics:
1.  **Platform:** Projects must be developed using the **Unity** engine and tagged for **VR**.
2.  **Popularity (Stars > 10):** We use stars as a proxy for community validation. A threshold of 50 stars filters out student course projects and personal experiments which lack industrial relevance.
3.  **Organizational Complexity (> 3 Contributors):** We distinguish between "Personal" and "Organizational" projects using a threshold of 3 developers. This ensures our dataset captures refactoring patterns driven by team collaboration and maintainability pressures.
4.  **Active Maintenance:** We prioritize the most recently updated projects to ensure the analysis reflects modern Unity VR development practices (e.g., XR Interaction Toolkit, URP) and avoids abandoned repositories.

## 1. Environment Setup
We utilize the `PyGithub` library to interact with the GitHub API. This requires a Personal Access Token (PAT) with `repo` read permissions to bypass rate limits and access metadata.

*Note: For security, the token is retrieved from Google Colab's "Secrets" manager.*

In [ ]:
# @title Install Dependencies & Connect to API
!pip install PyGithub pandas -q

import pandas as pd
from github import Github, Auth
from google.colab import userdata
import time
from datetime import datetime

# Securely fetch the token from Colab Secrets
try:
    token = userdata.get('GITHUB_TOKEN')
    print("✓ Token retrieved successfully.")
except Exception as e:
    print("✗ Error: Could not find 'GITHUB_TOKEN'. Please add it to the Secrets manager in the left sidebar.")

# Initialize the API Client
try:
    auth = Auth.Token(token)
    g = Github(auth=auth)
    user = g.get_user()
    print(f"✓ API Connected as: {user.login}")
    print(f"✓ Rate Limit Remaining: {g.get_rate_limit().rate.remaining}")
except Exception as e:
    print(f"✗ Connection Failed: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.5 MB/s eta 0:00:00
✓ Token retrieved successfully.
✓ API Connected as: Quorthon13
✓ Rate Limit Remaining: 4999


## 2. Mining & Filtering Strategy
The function below queries GitHub for `topic:unity` and `topic:vr`. It iteratively checks each result against our inclusion criteria.

**Filtering Logic:**
* `repo.stargazers_count < 10`: **Exclude.** (Low impact/toy project)
* `repo.archived == True`: **Exclude.** (Abandoned/Deprecated)
* `contributor_count > 3`: **Classify as "Organizational".**
* `contributor_count <= 3`: **Classify as "Personal".**

*Note: Determining the total commit count (to ensure >100 commits) is an expensive API operation. To optimize performance, we will defer the commit count validation to Phase 2 (Local Analysis), where it can be performed instantly after cloning.*

In [ ]:
import pandas as pd
import time
from github import GithubException

def get_vr_target_list():
    """
    Searches GitHub for Unity VR repositories filtering by Forks (C1),
    Maintenance (C2), and Activity (C3).
    """

    queries = [
        "language:C# topic:unity topic:vr",
        "language:C# topic:unity topic:virtual-reality",
        "language:C# topic:unity topic:ar",
        "language:C# topic:unity topic:augmented-reality",
        "language:C# topic:unity topic:xr",
        "language:C# topic:unity topic:extended-reality"
    ]

    projects_data = []
    processed_ids = set()

    print(f"🔎 Searching GitHub for targets (Filters: No Forks, Active, >100 Commits)...")
    print("-" * 110)
    print(f"{'Repository Name':<35} | {'Stars':<6} | {'Commits':<8} | {'Contributors':<12} | {'Status'}")
    print("-" * 110)

    for query in queries:
        # Search repositories sorted by recently updated
        repositories = g.search_repositories(query=query, sort="stars", order="desc")

        for repo in repositories:
            repo_name = repo.full_name[:34]

            try:
                # --- Deduplication ---
                if repo.id in processed_ids:
                    continue

                # --- C1: Forks Filter ---
                # Exclude forks to ensure original work
                if repo.fork:
                    print(f"{repo_name:<35} | {'-':<6} | {'-':<8} | {'-':<12} | ❌ Fork")
                    continue

                # --- C2: Maintenance Filter ---
                # Exclude archived repositories
                if repo.archived:
                    print(f"{repo_name:<35} | {'-':<6} | {'-':<8} | {'-':<12} | ❌ Archived")
                    continue

                # --- C3: Activity Filter (> 100 Commits) ---
                # Note: This consumes extra API calls and slows down execution
                try:
                    commit_count = repo.get_commits().totalCount
                    if commit_count < 100:
                        print(f"{repo_name:<35} | {repo.stargazers_count:<6} | {commit_count:<8} | {'-':<12} | ❌ < 100 Commits")
                        continue
                except GithubException:
                    # Handles empty repositories or API access issues
                    print(f"{repo_name:<35} | {'-':<6} | {'?':<8} | {'-':<12} | ⚠️ Commit fetch failed")
                    continue

                # --- Contributor Classification ---
                contributors = repo.get_contributors()
                contributor_count = 0
                try:
                    for _ in contributors:
                        contributor_count += 1
                        if contributor_count > 3:
                            break
                except Exception:
                    contributor_count = 1

                project_type = "Organizational" if contributor_count > 3 else "Personal"

                # --- Store Data ---
                processed_ids.add(repo.id)
                projects_data.append({
                    "name": repo.name,
                    "full_name": repo.full_name,
                    "url": repo.clone_url,
                    "stars": repo.stargazers_count,
                    "commits": commit_count,
                    "project_type": project_type,
                    "description": repo.description,
                    "created_at": repo.created_at.strftime("%Y-%m-%d"),
                    "last_updated": repo.updated_at.strftime("%Y-%m-%d")
                })

                print(
                    f"{repo.name[:34]:<35} | "
                    f"{repo.stargazers_count:<6} | "
                    f"{commit_count:<8} | "
                    f"{('>3' if contributor_count > 3 else str(contributor_count)):<12} | "
                    f"✅ Added"
                )

                # Sleep slightly longer due to heavier commit fetching
                time.sleep(0.5)

            except Exception as e:
                print(f"{repo_name:<35} | {'-':<6} | {'-':<8} | {'-':<12} | ❌ Error: {type(e).__name__}")
                continue

    print("-" * 110)
    print(f"🎉 Mining Complete. Found {len(projects_data)} repositories.")
    return pd.DataFrame(projects_data)

In [ ]:
# --- EXECUTION ---
from google.colab import drive
import os

# 1. Mount Google Drive
# This will trigger a popup asking for permission to access your Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Execute the miner
# We fetch the top 100 candidates based on our criteria
df_targets = get_vr_target_list()

# 3. Preview the Distribution
print("\nDataset Composition:")
print(df_targets['project_type'].value_counts())

# 4. Define Persistence Path
# This creates a folder named 'VR_Refactoring_Study' in your main Drive
folder_path = "/content/drive/My Drive/VR_Refactoring_Study"
os.makedirs(folder_path, exist_ok=True)

# 5. Save to Drive
timestamp = datetime.now().strftime("%Y%m%d")
filename = f"vr_repo_targets_{timestamp}.csv"
full_path = f"{folder_path}/{filename}"

df_targets.to_csv(full_path, index=False)

print(f"\n✅ Success! File saved permanently to: {full_path}")
print("You can now open the next notebook and load this file directly.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔎 Searching GitHub for targets (Filters: No Forks, Active, >100 Commits)...
--------------------------------------------------------------------------------------------------------------
Repository Name                     | Stars  | Commits  | Contributors | Status
--------------------------------------------------------------------------------------------------------------
VRTK                                | 3724   | 2473     | >3           | ✅ Added
InputSystem                         | 1497   | 2612     | >3           | ✅ Added
open-brush                          | 1058   | 851      | >3           | ✅ Added
EditorXR                            | 942    | 6176     | >3           | ✅ Added
VirtualMotionCapture                | 805    | 1064     | >3           | ✅ Added
UnityPlugin                         | 573    | 7722     | >3   

## 3. Next Steps (Phase 2)
The CSV file generated above serves as the input for **Phase 2: Repository Mining**.
In the next notebook, we will:
1.  Iterate through the `url` column.
2.  Clone each repository temporarily.
3.  Filter for `Total Commits > 100`.
4.  Mine the git history for refactoring keywords.